Esse notebook destina-se a análise dos dados referentes as propostas legislativas no período de 1934 a 2021. As perguntas estabelecidas e as hipóteses a serem comprovadas foram:

1. Houve aumento na quantidade anual de propostas legislativas cuja temática beneficia grupos historicamente marginalizados no período entre 1934 e 2021?
    - **Hipótese:** Sim, houve aumento na quantidade anual de propostas legislativas cuja temática beneficia grupos historicamente marginalizados.
2. Qual é o coeficiente de correlação entre a quantidade anual de propostas legislativas cuja temática beneficia grupos historicamente marginalizados e o percentual de mulheres eleitas para o cargo de deputada federal entre os anos de 1995 e 2019?
    - **Hipótese:** Fraco
    
Vale ressaltar que existem registros apenas de algumas proposições legislativas tramitadas entre os anos de 1934 e 1945, além disso os dados das proposições legislativas entre os anos de 1946 e 2000 estão parcialmente disponibilizados - estão abertos apenas os referentes às “proposições de tipos que poderiam se tornar (ou se tornaram) leis e normas jurídicas” (Brasil, [2021?]), sendo assim as análises realizadas com os dados desse período podem não refletir a realidade. Também, ressalta-se que não foi realizado um estudo aprofundado em Ciência da Informação para a criação de vocabulário controlado usado neste relatório, sendo assim as análises a seguir destinam-se apenas ao exercício proposto para este trabalho de conclusão de curso.

In [1]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

## 1. Houve aumento na quantidade anual de propostas legislativas cuja temática beneficia grupos historicamente marginalizados no período entre 1934 e 2021?
**Hipótese:** Sim, houve aumento na quantidade anual de propostas legislativas cuja temática beneficia grupos historicamente marginalizados.

In [ ]:
proposicoes = pd.read_csv('../dados/proposicoes_legislativas_limpas_classificadas.csv')

In [ ]:
proposicoes.head()

Agrupa por ano a quantidade de propostas legislativas referente a proteção de direitos de grupos historicamente marginalizados.

In [ ]:
proposicoes_anuais = proposicoes[['ano', 'temas_classificador']].groupby(by=['ano']).count()

In [ ]:
proposicoes_anuais.tail(10)

Visualiza em um plano cartesiano

In [ ]:
proposicoes_anuais.plot.line()

In [ ]:
proposicoes_anuais = proposicoes_anuais.reset_index()
proposicoes_anuais.to_csv('../dados/proposicoes_anuais.csv', index=False)

In [ ]:
print(f'''
Pergunta: Houve aumento na quantidade anual de propostas legislativas cuja temática beneficia grupos historicamente marginalizados no período entre 1934 e 2021? \n 
Resposta: Conforme observado no gráfico acima, apesar das oscilações, há uma tendência de crescimento positivo na quantidade de propostas apresentadas sobre a temática estudada. 
''')

## 2. Qual é o coeficiente de correlação entre a quantidade anual de propostas legislativas cuja temática beneficia grupos historicamente marginalizados e o percentual de mulheres eleitas para o cargo de deputada federal entre os anos de 1995 e 2019?
**Hipótese:** A correlação entre a quantidade anual de propostas legislativas cuja temática beneficia grupos historicamente marginalizados e o percentual de mulheres eleitas é baixa, ou seja, há um baixo grau de correlação.

Importa dados das análises constantes no notebook [analise-candidaturas-eleitas](/analise-candidaturas-eleitas.csv)

In [ ]:
analise_genero_1995_2007 = pd.read_csv('../dados/analise_genero_1995_2007.csv')
analise_genero_2007_2019 = pd.read_csv('../dados/analise_genero_2007_2019.csv')

Verifica se as colunas dos dataframes são identicas

In [ ]:
analise_genero_1995_2007.columns == analise_genero_2007_2019.columns

Cria dataframe único

In [ ]:
analise_genero_1995_2019 = pd.concat([analise_genero_1995_2007, analise_genero_2007_2019], ignore_index=True)

In [ ]:
analise_genero_1995_2019

In [ ]:
analise_genero_1995_2019.rename(columns={'Unnamed: 0': 'ano'}, inplace=True)

In [ ]:
analise_genero_1995_2019.drop(index=3, inplace=True)

In [ ]:
analise_genero_1995_2019

In [ ]:
anos = analise_genero_1995_2019['ano'].tolist()
anos.append(2021)
anos

Insere período completo de cada legislatura, levando em conta que proporcionalidade de gênero só é alterada a cada nova eleição.

In [ ]:
for ano in anos:
    mulher_percentual = analise_genero_1995_2019['mulher'][analise_genero_1995_2019['ano'] == ano].item()
    homem_percentual = analise_genero_1995_2019['homem'][analise_genero_1995_2019['ano'] == ano].item()
    if ano < 2021:
        dados = pd.DataFrame(data={
            'ano': [ano+1, ano+2, ano+3], 
            'mulher': [mulher_percentual, mulher_percentual, mulher_percentual], 
            'homem': [homem_percentual, homem_percentual, homem_percentual]}
                            )
        analise_genero_1995_2019 = pd.concat([analise_genero_1995_2019, dados])

In [ ]:
analise_genero_1995_2019.sort_values(by=['ano'], inplace=True)

In [ ]:
analise_genero_1995_2019.reset_index(drop=True, inplace=True)

In [ ]:
analise_genero_1995_2019.tail()

In [ ]:
analise_genero_1995_2019.drop(index=27, inplace=True)    

Insere total anual de propostas legislativas cuja temática beneficia grupos historicamente marginalizados.

In [ ]:
def insere_qnt_propostas(ano_candidaturas_eleitas):
    for ano, qnt_tema in zip(proposicoes_anuais['ano'], proposicoes_anuais['temas_classificador']):
        if ano == ano_candidaturas_eleitas:
            return qnt_tema

In [ ]:
analise_genero_1995_2019['qnt_proposicoes'] = analise_genero_1995_2019['ano'].apply(insere_qnt_propostas)

In [ ]:
analise_genero_1995_2019.head(10)

Criar matriz de correlação

In [ ]:
correlacao = analise_genero_1995_2019[['mulher', 'homem', 'qnt_proposicoes']].corr(method='pearson')

In [ ]:
coeficiente_correlacao_mulher_qnt_temas = round(correlacao['mulher']['qnt_proposicoes'],2)

In [ ]:
correlacao_matriz_triangular = np.triu(np.ones_like(correlacao))

In [ ]:
sns.heatmap(correlacao, annot=True, mask=correlacao_matriz_triangular)

In [ ]:
correlacao.to_csv('../dados/coeficiente_correlacao_mulher_qnt_temas.csv')

In [ ]:
print(f'''Qual é o coeficiente de correlação entre a quantidade anual de propostas legislativas cuja temática beneficia grupos historicamente marginalizados e o percentual de mulheres eleitas para o cargo de deputada federal entre os anos de 1995 e 2019?\n 
- Conforme observado na matriz de correlação acima, este coeficiente de correlação de Pearson é {coeficiente_correlacao_mulher_qnt_temas}, o que caracteriza uma fraca correlação entre os itens analisados.
''')